<a href="https://colab.research.google.com/github/dprugby/custom_CNN/blob/master/custom_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Custom ResNets with Keras

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from functools import partial

## A traditional CNN (as a benchmark on Fashion CIFAR10)

In [2]:
# import CIFAR10
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
X_valid, X_train = X_train_full[:10000] / 255., X_train_full[10000:] / 255.
y_valid, y_train = y_train_full[:10000], y_train_full[10000:]
X_test = X_test / 255.

print(X_train.shape, X_valid.shape, X_test.shape)
print(y_train.shape, y_valid.shape, y_test.shape)

170500096/170498071 [==============================] - 6s 0us/step
(40000, 32, 32, 3) (10000, 32, 32, 3) (10000, 32, 32, 3)
(40000, 1) (10000, 1) (10000, 1)


In [3]:
# assemble a simple CNN
model = keras.models.Sequential([
    keras.layers.Conv2D(64, 7, activation="relu", padding="same", input_shape=[32, 32, 3]),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation="softmax"),
])

In [24]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_146 (Conv2D)          (None, 32, 32, 64)        9472      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_147 (Conv2D)          (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_148 (Conv2D)          (None, 16, 16, 128)       147584    
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_149 (Conv2D)          (None, 8, 8, 256)         295168    
_________________________________________________________________
conv2d_150 (Conv2D)          (None, 8, 8, 256)        

In [25]:
# compile and train the model
optimizer = tf.keras.optimizers.Adagrad()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20
 751/1407 [===============>..............] - ETA: 11s - loss: 2.2991 - accuracy: 0.1114

KeyboardInterrupt: ignored

In [ ]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 1.2114 - accuracy: 0.5726


[1.2113527059555054, 0.5726000070571899]

## A shallow network with Inception blocks

In [26]:
# a custom residual unit, obtained by subclassing keras.layers.Layer
class Inception(keras.layers.Layer):
    def __init__(self, filters, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        self.activation = keras.activations.get(activation)
        self.layers_0 = [
        keras.layers.Conv2D(self.filters[0], 1, strides=1, padding="same", use_bias=False),
        self.activation]
        self.layers_1 = [
        keras.layers.Conv2D(self.filters[1], 1, strides=1, padding="same", use_bias=False),
        self.activation, 
        keras.layers.Conv2D(self.filters[2], 3, strides=1, padding="same", use_bias=False),
        self.activation]
        self.layers_2 = [
        keras.layers.Conv2D(self.filters[3], 1, strides=1, padding="same", use_bias=False),
        self.activation, 
        keras.layers.Conv2D(self.filters[4], 5, strides=1, padding="same", use_bias=False),
        self.activation]
        self.layers_3 = [
        keras.layers.MaxPooling2D(pool_size=2, strides=1, padding="same"), 
        keras.layers.Conv2D(self.filters[5], 5, strides=1, padding="same", use_bias=False),
        self.activation]
        
    def call(self, inputs):
        Z0 = inputs
        Z1 = inputs
        Z2 = inputs
        Z3 = inputs

        for layer in self.layers_0:
            Z0 = layer(Z0)
        for layer in self.layers_1:
            Z1 = layer(Z1)
        for layer in self.layers_2:
            Z2 = layer(Z2)
        for layer in self.layers_3:
            Z3 = layer(Z3)
        
        return tf.concat([Z0, Z1, Z2, Z3], axis=3)

In [27]:
# assemble a simple CNN
model = keras.models.Sequential([
    keras.layers.Conv2D(32, 5, activation="relu", padding="same", input_shape=[32, 32, 3]),
    keras.layers.MaxPooling2D(pool_size=3, strides=1, padding="same"),
    Inception([32, 48, 64, 8, 16, 16]),
    keras.layers.Conv2D(32, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(pool_size=3, strides=1, padding="same"),
    Inception([32, 48, 64, 8, 16, 16]),
    keras.layers.GlobalAveragePooling2D(),
    #keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation="softmax"),
])

In [28]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_151 (Conv2D)          (None, 32, 32, 32)        2432      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 32, 32, 32)        0         
_________________________________________________________________
inception_2 (Inception)      (None, 32, 32, 128)       46464     
_________________________________________________________________
conv2d_158 (Conv2D)          (None, 32, 32, 32)        36896     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 32, 32, 32)        0         
_________________________________________________________________
inception_3 (Inception)      (None, 32, 32, 128)       46464     
_________________________________________________________________
global_average_pooling2d_4 ( (None, 128)              

In [29]:
# compile and train the model
optimizer = tf.keras.optimizers.Adagrad()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid))

Epoch 1/100
 251/1407 [====>.........................] - ETA: 41s - loss: 2.3006 - accuracy: 0.1032

KeyboardInterrupt: ignored

## Mini ResNet

In [61]:
keras.layers.Conv2D(filters, kernel_size=3, strides=1,
                        padding="SAME", use_bias=False)

class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            keras.layers.Conv2D(filters, kernel_size=3, strides=strides,
                        padding="SAME", use_bias=False),
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.Conv2D(filters, kernel_size=3, strides=1,
                        padding="SAME", use_bias=False),
            keras.layers.BatchNormalization()]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                keras.layers.Conv2D(filters, kernel_size=1, strides=strides,
                        padding="SAME", use_bias=False),
                keras.layers.BatchNormalization()]

    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)

In [75]:
model = keras.models.Sequential()
model.add(DefaultConv2D(64, kernel_size=7, strides=2,
                        input_shape=[32, 32, 3]))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(ResidualUnit(64))
model.add(ResidualUnit(64))
model.add(keras.layers.Dropout(0.3))
model.add(ResidualUnit(128, strides=2))
model.add(ResidualUnit(128, strides=1))
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(10, activation="softmax"))

In [76]:
model.summary()

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_329 (Conv2D)          (None, 16, 16, 64)        9408      
_________________________________________________________________
batch_normalization_286 (Bat (None, 16, 16, 64)        256       
_________________________________________________________________
activation_24 (Activation)   (None, 16, 16, 64)        0         
_________________________________________________________________
residual_unit_118 (ResidualU (None, 16, 16, 64)        74240     
_________________________________________________________________
residual_unit_119 (ResidualU (None, 16, 16, 64)        74240     
_________________________________________________________________
dropout_9 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
residual_unit_120 (ResidualU (None, 8, 8, 128)       

In [77]:
# compile and train the model
optimizer = tf.keras.optimizers.Nadam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_valid, y_valid), batch_size=128)

Epoch 1/50
313/313 [==============================] - 20s 64ms/step - loss: 1.6230 - accuracy: 0.4122 - val_loss: 2.4043 - val_accuracy: 0.2878
Epoch 2/50
313/313 [==============================] - 20s 62ms/step - loss: 1.1739 - accuracy: 0.5791 - val_loss: 1.4072 - val_accuracy: 0.5067
Epoch 3/50
313/313 [==============================] - 20s 63ms/step - loss: 0.9743 - accuracy: 0.6557 - val_loss: 1.4470 - val_accuracy: 0.5300
Epoch 4/50
313/313 [==============================] - 20s 63ms/step - loss: 0.8419 - accuracy: 0.7054 - val_loss: 2.3335 - val_accuracy: 0.3344
Epoch 5/50
313/313 [==============================] - 20s 63ms/step - loss: 0.7394 - accuracy: 0.7418 - val_loss: 0.9104 - val_accuracy: 0.6712
Epoch 6/50
313/313 [==============================] - 20s 62ms/step - loss: 0.6622 - accuracy: 0.7693 - val_loss: 1.2565 - val_accuracy: 0.6331
Epoch 7/50
313/313 [==============================] - 19s 62ms/step - loss: 0.5954 - accuracy: 0.7941 - val_loss: 0.9855 - val_accuracy: